# Python file to Docker, to ECR, to Lambda 

## Manually Setup Docker Directory

lamdba_fxn_name/
├── Dockerfile
├── app/
│   ├── main.py          # Your main Lambda handler
│   ├── helper.py        # Your helper functions
│   └── requirements.txt # Python dependencies

## Custom Variables

In [1]:
lamdba_fxn_name = "nfl_matchups_model"
api_name = 'chalkjuice_api'

In [2]:
# List of AWS managed policy ARNs
managed_policies = [
    "arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator",
    "arn:aws:iam::aws:policy/AmazonAPIGatewayInvokeFullAccess",
    "arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs",
    "arn:aws:iam::aws:policy/AmazonAthenaFullAccess",
    "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
]

## Dependencies

In [3]:
import sys
import os
#!pip install dotenv
# Add the root of your project (one level up from "notebook")
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from packages.helpers.helpers import joel_boto

## Functions

In [4]:
def get_dependent_variables(lamdba_fxn_name):
    # Naviagte to the directory i want to build in (same name as lambda)
    path_docker = f"../docker/{lamdba_fxn_name}"

    # Get full IAM role ARN from the name
    role = jb.iam.get_role(RoleName=lamdba_fxn_name)
    role_arn = role["Role"]["Arn"]

    # Used to connect lamba to the correct ECR repo
    repo_uri = jb.get_ecr_repo_uri(lamdba_fxn_name)
    repo_image_uri = repo_uri + ':latest'


    return path_docker, role_arn, repo_uri, repo_image_uri

## Dependent Variables

In [5]:
# Connect to custom AWS class
jb = joel_boto()
apigw = jb.apigw
region = jb.region
account_id = jb.account_id
lambda_client = jb.lambda_client

running local credentials
✅ Logged in to ECR successfully.
✅ Connected to all clients successfully.


In [6]:
jb.create_lambda_iam_role(lamdba_fxn_name, managed_policies)

⚠️ IAM Role 'nfl_matchups_model' already exists.


In [7]:
# Create ECR Repository (one for each lambda fxn)
jb.create_ecr_repository(lamdba_fxn_name)

Repository 'nfl_matchups_model' already exists.


In [8]:
# Grab Dependent variables
path_docker, role_arn, repo_uri, repo_image_uri = get_dependent_variables(lamdba_fxn_name)

## Build, Push, and Create Lambda

In [18]:
# Build Docker image and push to ECR
jb.build_and_push_to_ECR(path_docker, repo_uri)

# Create a Lambda fxn using Image in ECR
jb.create_lambda_function(lamdba_fxn_name, role_arn, repo_image_uri)

⚠️ Lambda function 'nfl_matchups_model' already exists. Exiting.


## Update Lambda if exists to the latest image in ECR

In [19]:
# Update if already Exists
jb.update_lambda_function(lamdba_fxn_name, repo_image_uri)

✅ Lambda updated: 2025-06-15T01:03:39.000+0000


## API Gateway

### Create API

In [ ]:
# STEP 1: Create the WebSocket API
api = apigw.create_api(
    Name=api_name,
    ProtocolType='WEBSOCKET',
    RouteSelectionExpression='$request.body.action'
)
api_id = api['ApiId']

Created WebSocket API: 7aqddsnx56


### Create Routes and integrations

In [9]:
# List all lambda functions and their arns for copying into the next step
jb.list_active_lambdas()

Listing Lambda functions and ARNs:
Name: api1_disconnect, ARN: arn:aws:lambda:us-east-2:026090519913:function:api1_disconnect
Name: sleeper, ARN: arn:aws:lambda:us-east-2:026090519913:function:sleeper
Name: api1_connect, ARN: arn:aws:lambda:us-east-2:026090519913:function:api1_connect
Name: nfl_all_games, ARN: arn:aws:lambda:us-east-2:026090519913:function:nfl_all_games
Name: nfl_matchups_model, ARN: arn:aws:lambda:us-east-2:026090519913:function:nfl_matchups_model


In [10]:
# Replace from step above
lambda_routes = {
    '$connect':'arn:aws:lambda:us-east-2:026090519913:function:api1_connect',
    '$disconnect':'arn:aws:lambda:us-east-2:026090519913:function:api1_disconnect',
    'nfl_matchups_model': 'arn:aws:lambda:us-east-2:026090519913:function:nfl_matchups_model',
    'nfl_all_games': 'arn:aws:lambda:us-east-2:026090519913:function:nfl_all_games'
}

In [ ]:
jb.create_routes_and_integrations(lambda_routes, api_id)

### Create Permissions

In [ ]:
jb.create_lambda_permissions(lambda_routes, api_id, api_name)

Connected route '$connect' to Lambda
Connected route '$disconnect' to Lambda
Connected route 'nfl_matchups_model' to Lambda
Connected route 'nfl_all_games' to Lambda


#### Create Deployment

In [11]:
stage = apigw.create_deployment(ApiId=api_id)

NameError: name 'api_id' is not defined

#### Create Stage

In [ ]:
apigw.create_stage(
    ApiId=api_id,
    StageName='prod',
    DeploymentId=stage['DeploymentId']
)
print(f"Deployed API to 'prod' stage")

#### Update Stage

In [ ]:
deployment_id = stage['DeploymentId']
stage_name = 'prod'

apigw.update_stage(
    ApiId=api_id,
    StageName=stage_name,
    DeploymentId=deployment_id
)

{'ResponseMetadata': {'RequestId': '76830ab0-c6eb-4224-8f30-9b98e4fcb54b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 14 Jun 2025 12:14:00 GMT',
   'content-type': 'application/json',
   'content-length': '342',
   'connection': 'keep-alive',
   'x-amzn-requestid': '76830ab0-c6eb-4224-8f30-9b98e4fcb54b',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'MJ0dXE8siYcEBEw=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-684d6788-48a36d285be593056be612c2'},
  'RetryAttempts': 0},
 'CreatedDate': datetime.datetime(2025, 6, 14, 11, 12, 11, tzinfo=tzutc()),
 'DefaultRouteSettings': {'DataTraceEnabled': True,
  'DetailedMetricsEnabled': False,
  'LoggingLevel': 'ERROR'},
 'DeploymentId': 'ezacpq',
 'LastUpdatedDate': datetime.datetime(2025, 6, 14, 12, 14, tzinfo=tzutc()),
 'RouteSettings': {},
 'StageName': 'prod',
 'StageVariables': {},
 'Tags': {}}